In [1]:
%pip install --upgrade ipywidgets
%pip install --upgrade pip
%pip install numpy
%pip install pandas
%pip install scipy
%pip install sympy
%pip install transformers
%pip install SpeechRecognition
%pip install pygame
%pip install pyttsx3
%pip install torch
%pip install gtts
%pip install seaborn
%pip install streamlit
%pip install matplotlib
%pip install pygame
%pip install pyaudio

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
import os
import pygame
from gtts import gTTS
import speech_recognition as sr
import tempfile
import torch
import datetime
import requests
import random  # Import for random jokes/quotes
import time  # Import for reminders

# Suppress FutureWarning for tokenization spaces
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")



pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model_name = "facebook/blenderbot-400M-distill"# Larger model with the same functional capabilities facebook/blenderbot-3B # Qwen/Qwen2.5-1.5B-Instruct
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

WEATHER_API_KEY = '6375f15162952d140f9908db1d7ffb99'


In [4]:
def select_microphone():
    mic_index = None  # Automatically select microphone index 2
    print(f"Using microphone with index: {mic_index}")
    return mic_index


In [5]:
def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Lower energy threshold for faster detection
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)

        except sr.WaitTimeoutError:
            print("Listening timed out, please speak again.")
            return None  

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")  
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the Google Speech Recognition service; {e}")
        return None


In [6]:
def handle_time_date_queries(user_input):
    if "time" in user_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input or "today" in user_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    elif "location" in user_input or "where am i" in user_input:
        return get_current_location()  
    elif "weather" in user_input or "weather in my current location" in user_input:
        return get_current_weather()
    else:
        return None

def add_reminder(time_str, message_str):
    global reminders
    reminders_list = message_str.split('and')
    
    for reminder_message in reminders_list:
        reminder_message = reminder_message.strip()  
        if reminder_message:
            formatted_reminder = f"'{reminder_message}' at {time_str}"  
            reminders.append(formatted_reminder)
    
    return f"Added {len(reminders_list)} reminder(s): {', '.join([f'{msg.strip()} at {time_str}' for msg in reminders_list])}."

def list_reminders():
    if not reminders:
        return "You have no reminders set."
    
    reminder_str = ', '.join(reminders)
    return f"Your reminders are: {reminder_str}."

def delete_all_reminders():
    global reminders
    reminders.clear()
    return "All reminders have been deleted."



In [7]:
def get_current_location():
    try:
        response = requests.get("http://ip-api.com/json")
        data = response.json()
        
        if data['status'] == 'success':
            city = data.get('city', 'unknown city')
            region = data.get('regionName', 'unknown region')
            country = data.get('country', 'unknown country')
            return f"You are currently in {city}, {region}, {country}."
        else:
            return "Sorry, I could not determine your location."
    except Exception as e:
        return f"Error fetching location: {str(e)}"

def get_current_weather():
    location = get_current_location()
    if "in" in location:
        city = location.split("in ")[1].split(",")[0]
        try:
            weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
            response = requests.get(weather_url)
            data = response.json()
            if data['cod'] == 200:
                weather_description = data['weather'][0]['description']
                temperature = data['main']['temp']
                return f"The current weather in {city} is {weather_description} with a temperature of {temperature}°C."
            else:
                return "Sorry, I couldn't get the weather information."
        except Exception:
            return "Error fetching weather."
    else:
        return "I couldn't determine the location for weather."


In [8]:
reminders=[];
def add_reminder(time_str, message_str):
    global reminders
    reminders_list = message_str.split('and')
    
    for reminder_message in reminders_list:
        reminder_message = reminder_message.strip()  
        if reminder_message:
            formatted_reminder = f"'{reminder_message}' at {time_str}"  
            reminders.append(formatted_reminder)
    
    return f"Added {len(reminders_list)} reminder(s): {', '.join([f'{msg.strip()} at {time_str}' for msg in reminders_list])}."

def list_reminders():
    if not reminders:
        return "You have no reminders set."
    
    reminder_str = ', '.join(reminders)
    return f"Your reminders are: {reminder_str}."

def delete_all_reminders():
    global reminders
    reminders.clear()
    return "All reminders have been deleted."

In [9]:
def check_reminders():
    current_time = datetime.datetime.now().strftime("%H:%M")
    reminders_to_return = []
    for reminder in reminders:
        if reminder['time'] == current_time:
            reminders_to_return.append(reminder['message'])
    return reminders_to_return if reminders_to_return else None

In [10]:
def handle_calculator_query(user_input):
    try:
        expression = user_input.lower().replace('calculate', '').strip()
        result = eval(expression)
        return f"The result is {result}."
    except Exception as e:
        return "There was an error with your calculation."


In [11]:
jokes = [
    "Why don't scientists trust atoms? Because they make up everything!",
    "What do you get when you cross a snowman and a vampire? Frostbite!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "I told my wife she was drawing her eyebrows too high. She looked surprised!",
    "What’s orange and sounds like a parrot? A carrot!",
    "Why don’t skeletons fight each other? They don’t have the guts.",
    "Why can’t you give Elsa a balloon? Because she will let it go!",
    "Parallel lines have so much in common. It’s a shame they’ll never meet.",
    "Why don’t some couples go to the gym? Because some relationships don’t work out.",
    "I would tell you a construction pun, but I’m still working on it.",
    "I used to play piano by ear, but now I use my hands.",
    "Did you hear about the cheese factory that exploded? There was nothing left but de-brie.",
    "Why did the bicycle fall over? Because it was two-tired!",
    "I used to be addicted to the hokey pokey, but then I turned myself around.",
    "Why can’t your nose be 12 inches long? Because then it would be a foot!"
]

def tell_joke():
    if jokes:
        return jokes.pop(0)
    else:
        return "I’ve run out of jokes!"


In [12]:
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        tts.save(temp_audio_file.name)
        temp_file_path = temp_audio_file.name
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(temp_file_path)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.wait(50)  # Check every 50ms for faster termination

    except Exception as e:
        print(f"Error playing audio: {e}")
    finally:
        pygame.mixer.quit()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


In [13]:
def generate_response(user_input):
    user_input = user_input.lower()

    # Check if the user asked for the AI's name
    if "what is your name" in user_input:
        return "I am Amigo, your all-weather conversational companion, just like the bike that you are riding on."
    
    # Handle reminders
    if "reminder" in user_input:
        if "set" in user_input:
            try:
                parts = user_input.split("for")
                time = parts[1].split("to")[0].strip() if len(parts) > 1 else ""
                messages = parts[1].split("to")[1].strip() if len(parts) > 1 else ""

                if time and messages:
                    return add_reminder(time, messages)
                else:
                    return "Please specify the reminder time and message correctly."
            except:
                return "Please specify the reminder time and message correctly."
        elif "list" in user_input:
            return list_reminders()
        elif "delete all" in user_input or "remove all" in user_input:
            return delete_all_reminders()
    
    # Check reminders before generating response
    reminder_notifications = check_reminders()
    if reminder_notifications:
        return "You have the following reminders: " + ', '.join(reminder_notifications)

    # Handle calculator queries
    if "calculate" in user_input:
        return handle_calculator_query(user_input)

    # Handle jokes
    if "joke" in user_input:
        return tell_joke()
    
    # Handle time and date queries
    time_date_response = handle_time_date_queries(user_input)
    if time_date_response:
        return time_date_response

    # Tokenize user input
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, max_length=50)
    attention_mask = inputs['attention_mask']

    if 'attention_mask' not in inputs:
        inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])
    
    reply_ids = model.generate(inputs['input_ids'], max_length=50, pad_token_id=tokenizer.eos_token_id)
    bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    
    return bot_response


In [14]:
def listen_for_wake_word(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)

    print("Waiting for wake word 'Hey Amigo'...")
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source)
            
        try:
            user_input = recognizer.recognize_google(audio)
            if "hey amigo" in user_input.lower():
                print("Wake word detected!")
                return True
        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            print(f"Error with the Google Speech Recognition service: {e}")
            continue


In [15]:
def amigo_conversational_companion():
    mic_index = select_microphone()

    if listen_for_wake_word(mic_index):
        greeting = "Hi Hello Namaskara, How can I assist you today?"
        print(f"Amigo: {greeting}")
        speak_response(greeting)

        while True:
            user_input = capture_speech(mic_index)
            if user_input is None:
                continue

            if "exit" in user_input.lower() or "goodbye" in user_input.lower():
                farewell_message = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell_message}")
                speak_response(farewell_message)
                break

            bot_response = generate_response(user_input)
            print(f"Amigo: {bot_response}")
            speak_response(bot_response)


In [ ]:
if __name__ == "__main__":
    amigo_conversational_companion()


Using microphone with index: None
Waiting for wake word 'Hey Amigo'...
